In [1]:
from header import *
import scipy.constants as cnst

In [2]:
# --- constants --- 

#masses of nuclear particles:
        
#time unit in seconds 
T_unit=10**(-15)
L_unit=10**(-12)
E_unit=cnst.eV
M_unit=E_unit*T_unit**2/L_unit**2

assert(M_unit==E_unit*T_unit**2/L_unit**2)

#mass of the alpha particle
m1=(cnst.physical_constants['alpha particle mass'][0])/M_unit

#mass of the muon
m2=(cnst.physical_constants['muon mass'][0])/M_unit

#mass of the electron
m3=cnst.m_e/M_unit

m2=m3

#hbar
hbar=cnst.hbar/(E_unit*T_unit)

#elementry charge
e=cnst.e

#vacume permiativity
epsilon0=(cnst.physical_constants['vacuum electric permittivity'][0])*L_unit*E_unit



In [3]:
#prefactor
def pref(lambdas):
    return (
        lambdas[0]/np.sqrt(lambdas[3]*lambdas[6])
        *lambdas[1]/np.sqrt(lambdas[4]*lambdas[7])
        *lambdas[2]/np.sqrt(lambdas[5]*lambdas[8])
        )

#messy norm term
def mnt(x,y,z): 
    return (x**2*(y+z)+y**2*(x+z)+z**2*(x+y)+x*y*z)

def I110(lambdas):

    I=(
        0.5
        *pref(lambdas)
        *((2*lambdas[0]**2 + lambdas[0]*lambdas[1] + lambdas[0]*lambdas[2] + lambdas[1]*lambdas[2])
        /(
            np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
            *np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8])) 
        ))
        )
    
    return I

def I111(lambdas):

    I=(
        pref(lambdas)
        *(mnt(lambdas[0],lambdas[1],lambdas[2])
        /(
            np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
            *np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8])) 
        ))
        )
    return I

def I210(lambdas):
        
    I=(
        pref(lambdas)
        *((lambdas[0]**2*(3*lambdas[0]+2*lambdas[1]+lambdas[2])+lambdas[1]**2*(lambdas[0]+lambdas[2]) + lambdas[0]*lambdas[1]*lambdas[2])
        /(
            np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
            *np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8])) 
        ))
        )
    
    return I

def I300(lambdas):

    I=(
        3
        *pref(lambdas)
        *(lambdas[0]**2+lambdas[1]**2)/np.sqrt(mnt(lambdas[3],lambdas[4],lambdas[5]))
        *(lambdas[0]+lambdas[1])/np.sqrt(mnt(lambdas[6],lambdas[7],lambdas[8]))
        )

    return I

In [4]:
def N_func(i,j,alphas,betas,gammas):
    lambdas=[
        1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(betas[i])+betas[j]),1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(gammas[i])+gammas[j]),  1/(np.conjugate(betas[i])+betas[j]+np.conjugate(gammas[i])+gammas[j]),  
        0.5/(np.real(alphas[i])+np.real(betas[i])), 0.5/(np.real(alphas[i])+np.real(gammas[i])), 0.5/(np.real(betas[i])+np.real(gammas[i])),
         0.5/(np.real(alphas[j])+np.real(betas[j])), 0.5/(np.real(alphas[j])+np.real(gammas[j])), 0.5/(np.real(betas[j])+np.real(gammas[j]))
        ]
    I=I111(lambdas)
    
    return I

In [5]:
def order(first,second,third,lambdas):
    return [lambdas[first+second-1],lambdas[first+third-1],lambdas[second+third-1],
            lambdas[2+first+second],lambdas[2+first+third],lambdas[2+second+third],
            lambdas[5+first+second],lambdas[5+first+third],lambdas[5+second+third]
            ]

In [6]:

def H_func(i,j,alphas,betas,gammas):

	lambdas=[
		1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(betas[i])+betas[j]),1/(np.conjugate(alphas[i])+alphas[j]+np.conjugate(gammas[i])+gammas[j]),  1/(np.conjugate(betas[i])+betas[j]+np.conjugate(gammas[i])+gammas[j]),  
		0.5/(np.real(alphas[i])+np.real(betas[i])), 0.5/(np.real(alphas[i])+np.real(gammas[i])), 0.5/(np.real(betas[i])+np.real(gammas[i])),
		0.5/(np.real(alphas[j])+np.real(betas[j])), 0.5/(np.real(alphas[j])+np.real(gammas[j])), 0.5/(np.real(betas[j])+np.real(gammas[j]))
		]

	T=-0.5*hbar**2*(
			((alphas[j]**2+betas[j]**2)/m1  +  (alphas[j]**2+gammas[j]**2)/m2  +  (betas[j]**2+gammas[j]**2)/m3)*I111(lambdas)
			
			-2*alphas[j]*(1/m1+1/m2)*I110(order(2,1,0,lambdas))
			-2*betas[j]*(1/m1+1/m3)*I110(order(0,2,1,lambdas))  
			-2*gammas[j]*(1/m2+1/m3)*I110(lambdas) 
				
			+alphas[j]*betas[j]/m1*(I210(lambdas)+I210(order(1,0,2,lambdas))-I300(order(2,1,0,lambdas)))
			+alphas[j]*gammas[j]/m2*(I210(order(0,2,1,lambdas))+I210(order(2,0,1,lambdas))-I300(order(1,0,2,lambdas)))
			+betas[j]*gammas[j]/m3*(I210(order(1,2,0,lambdas))+I210(order(2,1,0,lambdas))-I300(lambdas))
			)
	
	
	V=(e**2/(4*np.pi*epsilon0))*(-2*I110(order(2,1,0,lambdas))-2*I110(order(0,2,1,lambdas))+I110(lambdas))

	return (T+V)

In [7]:
subspace=Subspace(2)

subspace.set_N_func(N_func)
subspace.set_H_func(H_func)

params=np.array([
    [[0.031],[0.02]],
    [[0.031],[0.02]],
    [[0],[0]]])*L_unit/10**(-12)

subspace.set_params(1,params)

subspace.make_N_mats()
subspace.make_H_mats()
subspace.find_N_eigens()
subspace.make_Y_mats()
subspace.make_invs_sqrt_beta_mats()
subspace.make_P_mats()
subspace.find_P_eigens()
subspace.find_energy_levels()

print(subspace.energy_levels)

Constructing the N matrices.
Constructing the H matrices.
Finding the eigenvectors and eigenvalues of the N matrices.
Constructing the Y matrices.
Constructing the inverse square root beta matrices.
Constructing the P matrices.
Finding P eigenvectors and eigenvalues.
Calculating the energy levels.
[[-77.45399442 -38.79659929]]


In [8]:
print(subspace.P_eigenvalues)

[[-38.79659929 -77.45399442]]


In [9]:
subspace.find_energy_eigenstates()
print(subspace.energy_eigenstates)

Calculating the components of the energy eigenstates.
[[[ 0.99800265 -0.06317202]
  [ 0.64776993 -0.76183602]]]


In [10]:
print(subspace.P_mats[0])# why not hermitian !!!!!!!!! :))))))))))))))))

[[-42.0488165   10.73424128]
 [ 10.72691829 -74.2017772 ]]


In [11]:
print(subspace.N_mats[0])

[[1.        0.8668301]
 [0.8668301 1.       ]]


In [12]:
print(subspace.H_mats[0])# why not hermitian

[[-77.41117992 -66.46306315]
 [-66.45941188 -66.71056808]]


In [13]:
subspace=Subspace(1)

subspace.set_N_func(N_func)
subspace.set_H_func(H_func)

params=np.array([
    [[0.031]],
    [[0.031]],
    [[0.0]]])

subspace.set_params(1,params)

subspace.make_N_mats()
subspace.make_H_mats()
subspace.find_N_eigens()
subspace.make_Y_mats()
subspace.make_invs_sqrt_beta_mats()
subspace.make_P_mats()
subspace.find_P_eigens()
subspace.find_energy_levels()

print(subspace.energy_levels)

Constructing the N matrices.
Constructing the H matrices.
Finding the eigenvectors and eigenvalues of the N matrices.
Constructing the Y matrices.
Constructing the inverse square root beta matrices.
Constructing the P matrices.
Finding P eigenvectors and eigenvalues.
Calculating the energy levels.
[[-77.41117992]]


In [ ]:

E=[]

for n in range(1,46):
    print(n)
    subspace=Subspace(n)

    subspace.set_N_func(N_func)
    subspace.set_H_func(H_func)

    param12=np.array([0.03,0.02,0.04,0.01,0.05,0.035,0.025])
    param13=np.array([0.03,0.02,0.04,0.01,0.05,0.035,0.025])
    param23=np.array([0,0.0045,0.004,0.005,0.003,0.006,0.002])

    params=np.expand_dims(np.array(np.meshgrid(param12,param13,[0],indexing='ij')).reshape(3,-1),-1)[:,:147]

    subspace.set_params(1,params)

    subspace.make_N_mats_vectorized()
    subspace.make_H_mats_vectorized()
    subspace.find_N_eigens()
    subspace.make_Y_mats()
    subspace.make_invs_sqrt_beta_mats()
    subspace.make_P_mats()
    subspace.find_P_eigens()
    subspace.find_energy_levels()

    E.append(subspace.energy_levels[:,0])
    
import matplotlib.pyplot as plt
plt.scatter(np.arange(len(E)),E)

plt.show()

1
Constructing the N matrices.
Constructing the H matrices.
Finding the eigenvectors and eigenvalues of the N matrices.
Constructing the Y matrices.
Constructing the inverse square root beta matrices.
Constructing the P matrices.
Finding P eigenvectors and eigenvalues.
Calculating the energy levels.
2
Constructing the N matrices.
Constructing the H matrices.
Finding the eigenvectors and eigenvalues of the N matrices.
Constructing the Y matrices.
Constructing the inverse square root beta matrices.
Constructing the P matrices.
Finding P eigenvectors and eigenvalues.
Calculating the energy levels.
3
Constructing the N matrices.
Constructing the H matrices.
Finding the eigenvectors and eigenvalues of the N matrices.
Constructing the Y matrices.
Constructing the inverse square root beta matrices.
Constructing the P matrices.
Finding P eigenvectors and eigenvalues.
Calculating the energy levels.
4
Constructing the N matrices.
Constructing the H matrices.
Finding the eigenvectors and eigenval

LinAlgError: Array must not contain infs or NaNs

In [ ]:
from scipy.optimize import minimize

def f(x):
    if np.any(x<=0):
        return 10000

    return H_func(0,0,[x[0]],[x[0]],[x[1]])

minimize(f,x0=[0.02,0.0045])



  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -80.28931854092394
        x: [ 3.091e-02  4.490e-03]
      nit: 5
      jac: [ 1.907e-06 -1.907e-06]
 hess_inv: [[ 6.577e-06 -6.049e-07]
            [-6.049e-07  3.556e-06]]
     nfev: 27
     njev: 9

In [ ]:
param12=np.array([0.03,0.02,0.04,0.01,0.05,0.035,0.025])
param13=np.array([0.03,0.02,0.04,0.01,0.05,0.035,0.025])
param23=np.array([0,0.0045,0.004,0.005,0.003,0.006,0.002][:3])

params=np.expand_dims(np.array(np.meshgrid(param12,param13,param23,indexing='ij')).reshape(3,-1),-1)

In [ ]:
print(params.shape)

(3, 147, 1)
